In [171]:
# importings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
%matplotlib notebook

In [172]:
# Reading Clean Data
covid_clean = pd.read_csv('../Data/Raw/owid-covid-data-clean.csv')

In [173]:
# First I want to convert Date to days that passed from start date:
covid_clean['date'] = pd.to_datetime(covid_clean['date'])
covid_clean['days_passed']= ((covid_clean['date']-covid_clean['date'].min()).dt.days + 1)

In [174]:
# Now because I create another policy for time, there is no need for date column:
covid_clean = covid_clean.drop('date', axis=1)

In [175]:
# Now I want to save data for each country individually and for each continent and for the world:
# But first for better view lets reorginze the columns:
covid_clean.columns

Index(['iso_code', 'continent', 'location', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'reproduction_rate', 'icu_patients',
       'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions',
       'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated',
       'total_boosters', 'new_vaccinations_smoothed',
       'new_people_vaccinated_smoothed', 'stringency_index',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index', 'population',
       'excess_mortality', 'total_cases_smoothed', 'total_deaths_smoothed',
       'total_boosters_smooth', 'people_fully_vaccinated_smooth',
       'total_vaccinations_smooth', 'total_people_vac

In [176]:
covid_clean = covid_clean[['iso_code', 'continent', 'location','days_passed', 'new_cases','total_cases',
       'new_cases_smoothed', 'total_cases_smoothed', 'new_deaths', 'total_deaths',
       'new_deaths_smoothed', 'total_deaths_smoothed','new_vaccinations_smoothed',
       'total_vaccinations','total_vaccinations_smooth',
       'new_people_vaccinated_smoothed', 'people_vaccinated', 'total_people_vaccinated_smooth',
       'people_fully_vaccinated', 'people_fully_vaccinated_smooth',
       'total_boosters', 'total_boosters_smooth',
       'reproduction_rate', 'stringency_index', 'excess_mortality',
       'icu_patients',
       'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index', 'population']]
covid_clean.head()

,iso_code,continent,location,days_passed,new_cases,total_cases,new_cases_smoothed,total_cases_smoothed,new_deaths,total_deaths,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population
0,AFG,Asia,Afghanistan,5,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,597.029,9.59,-1.0,-1.0,37.746,0.5,64.83,0.511,41128772.0
1,AFG,Asia,Afghanistan,6,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,597.029,9.59,-1.0,-1.0,37.746,0.5,64.83,0.511,41128772.0
2,AFG,Asia,Afghanistan,7,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,597.029,9.59,-1.0,-1.0,37.746,0.5,64.83,0.511,41128772.0
3,AFG,Asia,Afghanistan,8,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,597.029,9.59,-1.0,-1.0,37.746,0.5,64.83,0.511,41128772.0
4,AFG,Asia,Afghanistan,9,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,597.029,9.59,-1.0,-1.0,37.746,0.5,64.83,0.511,41128772.0


In [177]:
covid_clean = covid_clean.rename(columns={'people_vaccinated': 'total_people_vaccinated'})
covid_clean.columns

Index(['iso_code', 'continent', 'location', 'days_passed', 'new_cases',
       'total_cases', 'new_cases_smoothed', 'total_cases_smoothed',
       'new_deaths', 'total_deaths', 'new_deaths_smoothed',
       'total_deaths_smoothed', 'new_vaccinations_smoothed',
       'total_vaccinations', 'total_vaccinations_smooth',
       'new_people_vaccinated_smoothed', 'total_people_vaccinated',
       'total_people_vaccinated_smooth', 'people_fully_vaccinated',
       'people_fully_vaccinated_smooth', 'total_boosters',
       'total_boosters_smooth', 'reproduction_rate', 'stringency_index',
       'excess_mortality', 'icu_patients', 'hosp_patients',
       'weekly_icu_admissions', 'weekly_hosp_admissions', 'population_density',
       'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita',
       'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'handwashing_facilities',
       'hospital_beds_per_thousand', 'life_expectancy',
   

In [178]:
# Ok
# Before I save each country individually, lets look at datas, like Iran:


In [179]:
# You can see some of columns are static for each country,
# so I save these paremters in notebook to use them as a variable
# also It's obvious we don't need iso_code column so I'll delete it:
covid_clean = covid_clean.drop('iso_code', axis=1)

In [180]:
# Here I want to make data set for continents and world for calculations:
covid_clean['continent'].unique()

array(['Asia', 'Europe', 'Africa', 'Oceania', 'North America',
       'South America'], dtype=object)

In [181]:
# I have to say that for continets and world because some columns in not needed because of contex
# like static values and also stringency_index (depends on each country policy)
# and for excess_mortality and reproduction_rate I put mean of countris in that region for each day.
# for other columns I will sum the values.
# now let define table for each continent and world with initial value of 0:
unique_days = sorted(covid_clean['days_passed'].unique())
covid_clean_asia = pd.DataFrame({
    'days_passed': unique_days,
    'new_cases' : 0,'total_cases' : 0,
    'new_cases_smoothed' : 0, 'total_cases_smoothed' : 0,
    'new_deaths' : 0, 'total_deaths' : 0,
    'new_deaths_smoothed' : 0,'total_deaths_smoothed' : 0,
    'new_vaccinations_smoothed' : 0, 'total_vaccinations' : 0,
    'total_vaccinations_smooth' : 0,'new_people_vaccinated_smoothed' : 0,
    'total_people_vaccinated' : 0,'total_people_vaccinated_smooth' : 0,
    'people_fully_vaccinated' : 0, 'people_fully_vaccinated_smooth' : 0,
    'total_boosters' : 0, 'total_boosters_smooth' : 0, 
    'reproduction_rate' : 0, 'excess_mortality' : 0,
    'icu_patients' : 0, 'hosp_patients' : 0,
    'weekly_icu_admissions' : 0, 'weekly_hosp_admissions' : 0
})
covid_clean_europe = pd.DataFrame({
    'days_passed': unique_days,
    'new_cases' : 0,'total_cases' : 0,
    'new_cases_smoothed' : 0, 'total_cases_smoothed' : 0,
    'new_deaths' : 0, 'total_deaths' : 0,
    'new_deaths_smoothed' : 0,'total_deaths_smoothed' : 0,
    'new_vaccinations_smoothed' : 0, 'total_vaccinations' : 0,
    'total_vaccinations_smooth' : 0,'new_people_vaccinated_smoothed' : 0,
    'total_people_vaccinated' : 0,'total_people_vaccinated_smooth' : 0,
    'people_fully_vaccinated' : 0, 'people_fully_vaccinated_smooth' : 0,
    'total_boosters' : 0, 'total_boosters_smooth' : 0, 
    'reproduction_rate' : 0, 'excess_mortality' : 0,
    'icu_patients' : 0, 'hosp_patients' : 0,
    'weekly_icu_admissions' : 0, 'weekly_hosp_admissions' : 0
})
covid_clean_africa = pd.DataFrame({
    'days_passed': unique_days,
    'new_cases' : 0,'total_cases' : 0,
    'new_cases_smoothed' : 0, 'total_cases_smoothed' : 0,
    'new_deaths' : 0, 'total_deaths' : 0,
    'new_deaths_smoothed' : 0,'total_deaths_smoothed' : 0,
    'new_vaccinations_smoothed' : 0, 'total_vaccinations' : 0,
    'total_vaccinations_smooth' : 0,'new_people_vaccinated_smoothed' : 0,
    'total_people_vaccinated' : 0,'total_people_vaccinated_smooth' : 0,
    'people_fully_vaccinated' : 0, 'people_fully_vaccinated_smooth' : 0,
    'total_boosters' : 0, 'total_boosters_smooth' : 0, 
    'reproduction_rate' : 0, 'excess_mortality' : 0,
    'icu_patients' : 0, 'hosp_patients' : 0,
    'weekly_icu_admissions' : 0, 'weekly_hosp_admissions' : 0
})
covid_clean_oceania = pd.DataFrame({
    'days_passed': unique_days,
    'new_cases' : 0,'total_cases' : 0,
    'new_cases_smoothed' : 0, 'total_cases_smoothed' : 0,
    'new_deaths' : 0, 'total_deaths' : 0,
    'new_deaths_smoothed' : 0,'total_deaths_smoothed' : 0,
    'new_vaccinations_smoothed' : 0, 'total_vaccinations' : 0,
    'total_vaccinations_smooth' : 0,'new_people_vaccinated_smoothed' : 0,
    'total_people_vaccinated' : 0,'total_people_vaccinated_smooth' : 0,
    'people_fully_vaccinated' : 0, 'people_fully_vaccinated_smooth' : 0,
    'total_boosters' : 0, 'total_boosters_smooth' : 0, 
    'reproduction_rate' : 0, 'excess_mortality' : 0,
    'icu_patients' : 0, 'hosp_patients' : 0,
    'weekly_icu_admissions' : 0, 'weekly_hosp_admissions' : 0
})
covid_clean_north_america = pd.DataFrame({
    'days_passed': unique_days,
    'new_cases' : 0,'total_cases' : 0,
    'new_cases_smoothed' : 0, 'total_cases_smoothed' : 0,
    'new_deaths' : 0, 'total_deaths' : 0,
    'new_deaths_smoothed' : 0,'total_deaths_smoothed' : 0,
    'new_vaccinations_smoothed' : 0, 'total_vaccinations' : 0,
    'total_vaccinations_smooth' : 0,'new_people_vaccinated_smoothed' : 0,
    'total_people_vaccinated' : 0,'total_people_vaccinated_smooth' : 0,
    'people_fully_vaccinated' : 0, 'people_fully_vaccinated_smooth' : 0,
    'total_boosters' : 0, 'total_boosters_smooth' : 0, 
    'reproduction_rate' : 0, 'excess_mortality' : 0,
    'icu_patients' : 0, 'hosp_patients' : 0,
    'weekly_icu_admissions' : 0, 'weekly_hosp_admissions' : 0
})
covid_clean_south_america = pd.DataFrame({
    'days_passed': unique_days,
    'new_cases' : 0,'total_cases' : 0,
    'new_cases_smoothed' : 0, 'total_cases_smoothed' : 0,
    'new_deaths' : 0, 'total_deaths' : 0,
    'new_deaths_smoothed' : 0,'total_deaths_smoothed' : 0,
    'new_vaccinations_smoothed' : 0, 'total_vaccinations' : 0,
    'total_vaccinations_smooth' : 0,'new_people_vaccinated_smoothed' : 0,
    'total_people_vaccinated' : 0,'total_people_vaccinated_smooth' : 0,
    'people_fully_vaccinated' : 0, 'people_fully_vaccinated_smooth' : 0,
    'total_boosters' : 0, 'total_boosters_smooth' : 0, 
    'reproduction_rate' : 0, 'excess_mortality' : 0,
    'icu_patients' : 0, 'hosp_patients' : 0,
    'weekly_icu_admissions' : 0, 'weekly_hosp_admissions' : 0
})
covid_clean_world = pd.DataFrame({
    'days_passed': unique_days,
    'new_cases' : 0,'total_cases' : 0,
    'new_cases_smoothed' : 0, 'total_cases_smoothed' : 0,
    'new_deaths' : 0, 'total_deaths' : 0,
    'new_deaths_smoothed' : 0,'total_deaths_smoothed' : 0,
    'new_vaccinations_smoothed' : 0, 'total_vaccinations' : 0,
    'total_vaccinations_smooth' : 0,'new_people_vaccinated_smoothed' : 0,
    'total_people_vaccinated' : 0,'total_people_vaccinated_smooth' : 0,
    'people_fully_vaccinated' : 0, 'people_fully_vaccinated_smooth' : 0,
    'total_boosters' : 0, 'total_boosters_smooth' : 0, 
    'reproduction_rate' : 0, 'excess_mortality' : 0,
    'icu_patients' : 0, 'hosp_patients' : 0,
    'weekly_icu_admissions' : 0, 'weekly_hosp_admissions' : 0
})

In [182]:
def summing_datas (origin_data, target_data):
    columns_to_add = [  'new_cases',
                        'total_cases',
                        'new_cases_smoothed',
                        'total_cases_smoothed',
                        'new_deaths',
                        'total_deaths',
                        'new_deaths_smoothed',
                        'total_deaths_smoothed',
                        'new_vaccinations_smoothed',
                        'total_vaccinations',
                        'total_vaccinations_smooth',
                        'new_people_vaccinated_smoothed',
                        'total_people_vaccinated',
                        'total_people_vaccinated_smooth',
                        'people_fully_vaccinated',
                        'people_fully_vaccinated_smooth',
                        'total_boosters',
                        'total_boosters_smooth',
                        'reproduction_rate',
                        'excess_mortality',
                        'icu_patients',
                        'hosp_patients',
                        'weekly_icu_admissions',
                        'weekly_hosp_admissions']
    origin_data.set_index('days_passed', inplace=True)
    target_data.set_index('days_passed', inplace=True)
    # Update target_data with the sum of values from origin_data
    target_data += origin_data[columns_to_add]
    # Reset index
    target_data.reset_index(inplace=True)
    origin_data.reset_index(inplace=True)
def final_country_split (country_data, country_name):
    unique_values = country_data[['population_density',
       'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita',
       'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'handwashing_facilities',
       'hospital_beds_per_thousand', 'life_expectancy',
       'human_development_index', 'population']].drop_duplicates()
    drop_columns = ['continent', 'location', 'population_density',
       'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita',
       'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'handwashing_facilities',
       'hospital_beds_per_thousand', 'life_expectancy',
       'human_development_index', 'population']
    country_data = country_data.drop(drop_columns, axis = 1)
    unique_values.to_csv(f'../Data/Splited/Countries/owid-covid-data-clean-{country_name}-unique_values.csv', index=False)
    country_data.to_csv(f'../Data/Splited/Countries/owid-covid-data-clean-{country_name}.csv', index=False)

In [183]:
import logging

In [184]:
for country_name in covid_clean['location'].unique():
    country_data = covid_clean[covid_clean['location'] == country_name].copy()
    continent_name = country_data['continent'].unique().tolist()[0]
    if continent_name == 'Asia':
        summing_datas(country_data, covid_clean_asia)
    elif continent_name == "Europe":
        summing_datas(country_data, covid_clean_europe)
    elif continent_name == "Africa":
        summing_datas(country_data, covid_clean_africa)
    elif continent_name == "Oceania":
        summing_datas(country_data, covid_clean_oceania)
    elif continent_name == "North America":
        summing_datas(country_data, covid_clean_north_america)
    elif continent_name == "South America":
        summing_datas(country_data, covid_clean_south_america)
    summing_datas(country_data,covid_clean_world)
    final_country_split(country_data, country_name)

In [186]:
# now for this columns of continents and world we have to calculate the min of them
#: (reproduction_rate, excess_mortality)
#Asia
number_of_countries = len(covid_clean[covid_clean['continent'] == 'Asia']) 
covid_clean_asia = covid_clean_asia.fillna(0)
covid_clean_asia[['reproduction_rate', 'excess_mortality']] = covid_clean_asia[['reproduction_rate', 'excess_mortality']] / number_of_countries
# Europe
number_of_countries = len(covid_clean[covid_clean['continent'] == 'Europe']) 
covid_clean_europe = covid_clean_europe.fillna(0)
covid_clean_europe[['reproduction_rate', 'excess_mortality']] = covid_clean_europe[['reproduction_rate', 'excess_mortality']] / number_of_countries
# Africa
number_of_countries = len(covid_clean[covid_clean['continent'] == 'Africa']) 
covid_clean_africa = covid_clean_africa.fillna(0)
covid_clean_africa[['reproduction_rate', 'excess_mortality']] = covid_clean_africa[['reproduction_rate', 'excess_mortality']] / number_of_countries
# Oceania
number_of_countries = len(covid_clean[covid_clean['continent'] == 'Oceania']) 
covid_clean_oceania = covid_clean_oceania.fillna(0)
covid_clean_oceania[['reproduction_rate', 'excess_mortality']] = covid_clean_oceania[['reproduction_rate', 'excess_mortality']] / number_of_countries
# North America
number_of_countries = len(covid_clean[covid_clean['continent'] == 'North America']) 
covid_clean_north_america = covid_clean_north_america.fillna(0)
covid_clean_north_america[['reproduction_rate', 'excess_mortality']] = covid_clean_north_america[['reproduction_rate', 'excess_mortality']] / number_of_countries
# South America
number_of_countries = len(covid_clean[covid_clean['continent'] == 'South America']) 
covid_clean_south_america = covid_clean_south_america.fillna(0)
covid_clean_south_america[['reproduction_rate', 'excess_mortality']] = covid_clean_south_america[['reproduction_rate', 'excess_mortality']] / number_of_countries
# world
number_of_countries = len(covid_clean) 
covid_clean_world = covid_clean_world.fillna(0)
covid_clean_world[['reproduction_rate', 'excess_mortality']] = covid_clean_world[['reproduction_rate', 'excess_mortality']] / number_of_countries

In [187]:
# Saving Continents datas:
covid_clean_asia.to_csv(f'../Data/Splited/Continents/owid-covid-data-clean-Asia.csv', index=False)
covid_clean_europe.to_csv(f'../Data/Splited/Continents/owid-covid-data-clean-Europe.csv', index=False)
covid_clean_africa.to_csv(f'../Data/Splited/Continents/owid-covid-data-clean-Africa.csv', index=False)
covid_clean_oceania.to_csv(f'../Data/Splited/Continents/owid-covid-data-clean-Oceania.csv', index=False)
covid_clean_north_america.to_csv(f'../Data/Splited/Continents/owid-covid-data-clean-North America.csv', index=False)
covid_clean_south_america.to_csv(f'../Data/Splited/Continents/owid-covid-data-clean-South America.csv', index=False)
# Saving World datas:
covid_clean_world.to_csv(f'../Data/Splited/World/owid-covid-data-clean-World.csv', index=False)